### Load Random Objects from `Google Scanned Objects` Dataset

In [1]:
import sys
import random
import xml.etree.ElementTree as ET
sys.path.append('../package/mujoco_helper/') # for 'mujoco_parser'
sys.path.append('../package/kinematics_helper/') # for 'transforms'
sys.path.append('../package/utility/') # for 'utils'
from mujoco_parser import *
from utils import *
print ("MuJoCo version:",MUJOCO_VERSION)

MuJoCo version: (3, 1, 6)


In [2]:
# Get (Google's) scanned object names
folder_path = 'converted_models/'
scanned_object_names = [f.name for f in os.scandir(folder_path) if f.is_dir()]
print ("scanned_object_names:[%d]"%(len(scanned_object_names)))

# Sample object names
n_loaded_object = 10
loaded_objects = random.sample(scanned_object_names,n_loaded_object) # sample
print ("n_loaded_object:[%d]"%(n_loaded_object))
print ("loaded_objects:%s"%(loaded_objects))

# Append sampled object names to xml
xml_path = '../asset/scene_table.xml'
tree = ET.parse(xml_path)
root = tree.getroot()
for obj_name in loaded_objects:
    file_path = '%s/%s/model.xml'%(folder_path,obj_name)
    include_tag = ET.Element('include',attrib={'file':file_path})
    root.append(include_tag)
xml_string = prettify(root) # indent xml

# Print xml string
print ("\n\n",xml_string,"\n\n")

# Parse 'xml_string'
env = MuJoCoParserClass(name='Tabletop',xml_string=xml_string,verbose=True)

scanned_object_names:[1030]
n_loaded_object:[10]
loaded_objects:['Office_Depot_Dell_Series_1_Remanufactured_Ink_Cartridge_TriColor', 'Kanex_MultiSync_Wireless_Keyboard', 'ClimaCool_Aerate_2_W_Wide', 'Saccharomyces_Boulardii_MOS_Value_Size', 'PINEAPPLE_MARACA_6_PCSSET', 'BRAILLE_ALPHABET_AZ', 'Shurtape_Tape_Purple_CP28', 'ASICS_GELChallenger_9_Royal_BlueWhiteBlack', 'Reebok_ZIGLITE_RUSH', 'Shurtape_Gaffers_Tape_Silver_2_x_60_yd']


 <?xml version="1.0" ?>
<mujoco model="Scanned Objects">
    <size memory="500M"/>
    <compiler angle="radian" autolimits="true"/>
    <option integrator="RK4" noslip_iterations="20"/>
    <default>
        <joint armature="0.1" damping="0.1"/>
        <default class="visual_only">
            <geom type="mesh" contype="0" conaffinity="0" group="2"/>
        </default>
        <default class="no_visual_collision">
            <geom type="capsule" solimp=".9 .99 .001" solref=".015 1" group="3"/>
        </default>
        <default class="visual_collision">
  

In [3]:
env.reset()
env.init_viewer(
    black_sky   = True,
    transparent = False,
    geomgroup_0     = 1, # floor sky
    geomgroup_1     = 1, # collision
    geomgroup_2     = 1, # visual (this slows down)
)
obj_body_names = env.get_body_names(prefix='body_obj_')
obj_xyzs = sample_xyzs(
    n_sample=len(obj_body_names),
    x_range=[-0.4,+1.4],y_range=[-0.5,+0.5],z_range=[0.9,0.9],
    min_dist=0.25, # 0.25
)
for obj_idx,obj_body_name in enumerate(obj_body_names):
    env.set_p_base_body(body_name=obj_body_name,p=obj_xyzs[obj_idx,:])
    R_rand = rpy2r(np.deg2rad([0,0,360*np.random.rand()]))
    env.set_R_base_body(body_name=obj_body_name,R=R_rand)
env.forward(increase_tick=False)
while env.is_viewer_alive():
    env.step()
    if env.loop_every(tick_every=10):
        env.plot_T()
        env.plot_time()
        # env.plot_contact_info(rate=1.0)
        # env.plot_bodies_T(body_names=obj_body_names,plot_axis=False,axis_len=0.1,axis_width=0.005,plot_name=True)
        env.render()
print ("Done.")


Done.
